# Retail SVD Recommender

In this example we use SQL to calculate dot products for product recommmendations. Kinetica SQL gets converted to native GPU accelerated operations.

### Steps

1. Get the actual ratings from the rating table.
1. Use dot-products to compute the approximated ratings.
1. Take the difference of the actual and approximation to get the recomendation.

### Prerequsites

You should have run the [svd_calc.ipynb](svd_calc.ipynb) notebook to generate the vector tables.

In [ ]:
# Local libraries should automatically reload
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import sys 
sys.path.append('../KJIO')
import numpy as np
import pandas as pd
%aimport kodbc_io

pd.options.display.max_colwidth = 100

### Get actual purchases

Here we get the actual ratings of a customer. You can do this for any customer ID.

In [11]:
CUSTOMER_ID = 19434

_actual_df = kodbc_io.get_df("""
select
    am.product_parent,
    ap.product_title,
    am.star_rating
from amazon_matrix am
join amazon_products ap
    on am.product_parent = ap.product_parent
where am.customer_id = {}
order by am.product_parent
""".format(CUSTOMER_ID))

_actual_df = _actual_df.set_index('product_parent')
_actual_df

Connected to GPUdb ODBC Server (6.2.0.12.20180720232954)
Rows returned: 1


,product_title,star_rating
product_parent,,
342671732,DEWALT DW715 15-Amp 12-Inch Single-Bevel Compound Miter Saw,5


### Get approximated ratings

Earlier we computed $\sqrt{\Sigma}\tilde{\mathbf{U}}$ and $\sqrt{\Sigma}\tilde{\mathbf{V}}^T$ which were saved to tables. Here we use SQL to compute the rating approximation for a single customer:

$\mathbf{\tilde{a}}_i = \mathbf{\tilde{u}}_i \Sigma \mathbf{\tilde{V}}^T$

This reduces to a set of dot products:

$\mathbf{\tilde{a}}_i = (\sqrt{\Sigma} \mathbf{\tilde{u}}_i) \cdot (\sqrt{\Sigma} \mathbf{\tilde{v}}_i^T) $

In [14]:
_approx_df = kodbc_io.get_df("""
select top 10 
    iv.product_parent,
    ap.product_title,
    (cv.U0 * iv.V0)
    + (cv.U1 * iv.V1) 
    + (cv.U2 * iv.V2) 
    + (cv.U3 * iv.V3) 
    + (cv.U4 * iv.V4) 
    + (cv.U5 * iv.V5) 
    + (cv.U6 * iv.V6) 
    + (cv.U7 * iv.V7) 
    + (cv.U8 * iv.V8) 
    + (cv.U9 * iv.V9) 
    as item_rating
from svd_cust_vec as cv, svd_item_vec as iv
join amazon_products ap
    on ap.product_parent = iv.product_parent
where cv.customer_id = {}
order by item_rating desc
""".format(CUSTOMER_ID))

_approx_df = _approx_df.set_index('product_parent')

Connected to GPUdb ODBC Server (6.2.0.12.20180720232954)
Rows returned: 10


,product_title,item_rating
product_parent,,
672479254,CH Hanson 0 Stud 4 Sure Magnetic Stud Finder,0.020658
497942344,SE MH1047L Illuminated Multi-Power LED Head Magnifier,0.015847
359191662,SE 7503SD 3-Piece Power Extension Bit Set for Drills,0.014154
655034668,CLC Custom Leathercraft 125 Handyman Flex Grip Work Gloves,0.013996
655034668,"CLC Custom Leathercraft 125M Handyman Flex Grip Work Gloves, Medium",0.013996
101240200,Streamlight Nano Light Miniature Keychain LED Flashlight,0.013928
672080706,SE MZ101B Helping Hand with Magnifying Glass,0.013635
114878519,Mirka Bulldog Gold 5-Inch 8-Hole 60 Grit Grip Vacuum Discs,0.009574
859114292,Dremel 4486 MultiPro Keyless Chuck,0.007828


### Get recommended purchases

Here we generate recommendations which is the difference between the actual and apprximation.

$
\mathbf{\tilde{r}}_i = \mathbf{\tilde{a}}_i - \mathbf{a}_i
$

Consider that the approximation will contain the customers own ratings combined with other ratings from similar customers. When we subtract the customers actual ratings what remains are preferences from other similar customers.

These results are sorted by highest rating so the most significant recommendations are listed at the top.

In [15]:
_recommended_df = _approx_df.loc[set(_approx_df.index) - set(_actual_df.index)]
_recommended_df.sort_values('item_rating', ascending=False).head(4)

,product_title,item_rating
product_parent,,
672479254,CH Hanson 0 Stud 4 Sure Magnetic Stud Finder,0.020658
497942344,SE MH1047L Illuminated Multi-Power LED Head Magnifier,0.015847
359191662,SE 7503SD 3-Piece Power Extension Bit Set for Drills,0.014154
655034668,CLC Custom Leathercraft 125 Handyman Flex Grip Work Gloves,0.013996
